In [1]:
using SymPy
using LinearAlgebra
# 
using Parameters

In [2]:
import PyCall
PyCall.pyimport_conda("sympy.physics.quantum.spin", "sympy")
import_from(sympy.physics.quantum.spin, (:WignerD, :wignerd), typ=:Any)
PyCall.pyimport_conda("sympy.physics.wigner",       "sympy")
import_from(sympy.physics.wigner)

In [222]:
M = [1,-1,-1,-1];
gperp(ν,μ,l) = (ν==μ ? M[ν] : 0) - l[ν]*l[μ]/sum(l.*M.*l)
# 
function ϵ(α,μ,ν,β)
    length(Set([α,μ,ν,β]))!=4 && return 0
    inds = [α,μ,ν,β]
    prod(sign(inds[i]-inds[j]) for i in 1:4 for j in i+1:4)
end;
# ϵ(2,1,3,4),ϵ(1,2,3,4),ϵ(2,3,4,1),ϵ(3,4,1,1)

In [256]:
γ,E,γβ,k,p,θ1,θ2,ϕ = @vars γ E γβ k p θ_1 θ_2 ϕ positive=true
# 
k1 = [γ*E+γβ*k*cos(θ1), k*sin(θ1),0, γ*k*cos(θ1) + γβ*E]
k2 = [γ*E-γβ*k*cos(θ1),-k*sin(θ1),0,-γ*k*cos(θ1) + γβ*E]
# 
p1 = [γ*E+γβ*p*cos(θ2),  p*sin(θ2)*cos(ϕ),-p*sin(θ2)*sin(ϕ),  -(γ*p*cos(θ2) + γβ*E)]
p2 = [γ*E-γβ*p*cos(θ2), -p*sin(θ2)*cos(ϕ), p*sin(θ2)*sin(ϕ), -(-γ*p*cos(θ2) + γβ*E)]
#
q = (k1+k2)-(p1+p2)
# 
vv = simplify(expand(simplify(sum(k1 .* M .* k2).subs(k,E)).subs(γβ^2,γ^2-1)))
vv != simplify(expand(simplify(sum(p1 .* M .* p2).subs(p,E)).subs(γβ^2,γ^2-1))) && error("!!")
# vv
q'

1×4 Adjoint{Any,Array{Sym,1}}:
 0  0  0  4*E*γβ

# Covariant amplitude
$$
\Large
\begin{align}
A^\alpha = \epsilon^{\alpha,\mu,\rho,\tau}
    \quad\tilde{q}_{\tau}
    \quad \bar{v}(k_2)\gamma^\nu u(k_1)\,\,g_{\nu\mu}^{\perp}(k)
    \quad \bar{v}(p_2)\gamma^\sigma u(p_1)\,\,g_{\sigma\rho}^{\perp}(p)
\end{align}
$$
with $k = k_1+k_2$, $p = p_1+p_2$, and $\tilde{q} = k - p$

In [248]:
L(μ,ρ,v1,v2) = v1[μ]*v2[ρ]+v2[μ]*v1[ρ]-(μ==ρ ? M[μ] : 0) * vv # * sum(v1 .* M .* v2)
# 
L1rel = [expand(sum(L(ν,τ,k1,k2) * M[ν] * M[τ] *
                gperp(ν,μ,k1+k2) * gperp(τ,ρ,k1+k2)
                    for ν=1:4, τ=1:4)).subs(γβ^2,γ^2-1) for μ=1:4, ρ=1:4];
L2rel = [expand(sum(L(ν,τ,p1,p2) * M[ν] * M[τ] *
                gperp(ν,μ,p1+p2) * gperp(τ,ρ,p1+p2)
                    for ν=1:4, τ=1:4)).subs(γβ^2,γ^2-1) for μ=1:4, ρ=1:4];

In [249]:
simplify(sum(gperp(i,i,p1+p2)*M[i] for i in 1:4).subs(γβ^2,γ^2-1))

3

In [253]:
a = sum(L1rel[i,i]*M[i] for i in 1:4)
simplify(expand(simplify(a.subs(k,E)).subs(γβ^2,γ^2-1)))
# 
a = sum(L2rel[i,i]*M[i] for i in 1:4)
simplify(expand(simplify(a.subs(p,E)).subs(γβ^2,γ^2-1)))
# 

    2
-4*E 

In [254]:
cal = simplify(
    sum(gperp(α,α′,p1+p2+k1+k2)*gperp(α,α′′,p1+p2+k1+k2)*
        ϵ(α′,μ,ν,4)*
        ϵ(α′′,ρ,τ,4)*L1rel[μ,ρ]*L2rel[ν,τ]*
    M[α]*M[μ]*M[ν]*M[ρ]*M[τ]
    for α=1:4,α′=1:4,α′′=1:4,μ=1:4,ν=1:4,ρ=1:4,τ=1:4))

      4  2       4      2  2  2    2           2  2  2      2  2    2         
- 16*E *γ  + 16*E  - 4*E *k *γ *sin (θ_1) + 8*E *k *γ  + 4*E *k *sin (θ_1) - 8

  2  2      2  2  2    2           2  2  2      2  2    2           2  2      
*E *k  - 4*E *p *γ *sin (θ_2) + 8*E *p *γ  + 4*E *p *sin (θ_2) - 8*E *p  + 8*k

2  2  2    2         2           2  2  2    2           2  2  2    2          
 *p *γ *sin (θ_1)*sin (θ_2) - 4*k *p *γ *sin (θ_1) - 4*k *p *γ *sin (θ_2) + 8*

 2  2   2                                                 2  2    2         2 
k *p *γβ *sin(θ_1)*sin(θ_2)*cos(θ_1)*cos(θ_2)*cos(ϕ) - 8*k *p *sin (θ_1)*sin (

          2  2    2           2  2    2     
θ_2) + 4*k *p *sin (θ_1) + 4*k *p *sin (θ_2)

In [255]:
factor(expand(expand(expand(cal.subs(γβ^2,γ^2-1).subs(k,E).subs(p,E))/E^4)))

                  /   2         2           2                                 
8*(γ - 1)*(γ + 1)*\sin (θ_1)*sin (θ_2) - sin (θ_1) + sin(θ_1)*sin(θ_2)*cos(θ_1

                       2     \
)*cos(θ_2)*cos(ϕ) - sin (θ_2)/